In [1]:
# !pip3 install -U torch_geometric
# !pip3 install -U torch-scatter
# !pip3 install -U torch-sparse
# !pip3 install -U pandas
# !pip3 install -U numpy
# !pip3 install -U matplotlib
# !pip3 install -U sklearn
# !pip3 install -U PyTDC
# !pip3 install -U pre-commit
# !pip3 install -U plotly
# !pip3 install -U tdc
# !pip3 install -U ogb
# !pip3 install ogb==1.2.6

In [2]:
from benchmark import *

In [3]:
dataset = pd.read_csv(
    f"../dataset/{'_'.join(dataset_name.split('-'))}/mapping/mol.csv.gz"
)
X = dataset["smiles"]
y = dataset["HIV_active"]

In [4]:
n_molecules = X.shape[0]
n_molecules

41127

## Morgan Fingerprint

In [7]:
morgan_emf_times = [
    [
        [
            get_times_emf(
                X,
                MorganFingerprint,
                sparse=sparse,
                count=count,
                n_jobs=n_cores,
            )
            for n_cores in N_CORES
        ]
        for sparse in SPARSE_TYPES
    ]
    for count in COUNT_TYPES
]

KeyboardInterrupt: 

In [10]:
generator = fpgens.GetMorganGenerator()
print(type(generator))

morgan_rdkit_times = [
    [
        get_generator_times_rdkit(
            X,
            generator,
            sparse=sparse,
            count=count,
        )
        for sparse in SPARSE_TYPES
    ]
    for count in COUNT_TYPES
]

<class 'rdkit.Chem.rdFingerprintGenerator.FingeprintGenerator64'>


KeyboardInterrupt: 

In [ ]:
for count, i in enumerate(COUNT_TYPES):
    for sparse, j in enumerate(SPARSE_TYPES):
        plot_results(
            morgan_emf_times[i][j],
            morgan_rdkit_times[i][j],
            "Morgan Fingerprint",
            count,
            sparse,
            save=False,
        )

## Atom Pair

In [ ]:
atom_pair_emf_times = [
    [
        [
            get_times_emf(
                X,
                AtomPairFingerprint,
                sparse=sparse,
                count=count,
                n_jobs=n_cores,
            )
            for n_cores in N_CORES
        ]
        for sparse in SPARSE_TYPES
    ]
    for count in COUNT_TYPES
]

In [ ]:
generator = fpgens.GetAtomPairGenerator()

atom_pair_rdkit_times = [
    [
        get_generator_times_rdkit(
            X,
            generator,
            sparse=sparse,
            count=count,
        )
        for sparse in SPARSE_TYPES
    ]
    for count in COUNT_TYPES
]

In [ ]:
for count, i in enumerate(COUNT_TYPES):
    for sparse, j in enumerate(SPARSE_TYPES):
        plot_results(
            atom_pair_emf_times[i][j],
            atom_pair_rdkit_times[i][j],
            "Atom Pair Fingerprint",
            count,
            sparse,
            save=False,
        )

## Topological Torsion

In [ ]:
topological_torsion_emf_times = [
    [
        [
            get_times_emf(
                X,
                AtomPairFingerprint,
                sparse=sparse,
                count=count,
                n_jobs=n_cores,
            )
            for n_cores in N_CORES
        ]
        for sparse in SPARSE_TYPES
    ]
    for count in COUNT_TYPES
]

In [ ]:
generator = fpgens.GetTopologicalTorsionGenerator()

topological_torsion_rdkit_times = [
    [
        get_generator_times_rdkit(
            X,
            generator,
            sparse=sparse,
            count=count,
        )
        for sparse in SPARSE_TYPES
    ]
    for count in COUNT_TYPES
]

In [ ]:
for count, i in enumerate(COUNT_TYPES):
    for sparse, j in enumerate(SPARSE_TYPES):
        plot_results(
            topological_torsion_emf_times[i][j],
            topological_torsion_rdkit_times[i][j],
            "Topological Torsion Fingerprint",
            count,
            sparse,
            save=False,
        )

## MACCS Keys

In [36]:
MACCSKeys_emf_times = [
    [
        get_times_emf(X, MACCSKeysFingerprint, n_jobs=n_cores, sparse=sparse)
        for n_cores in N_CORES
    ]
    for sparse in SPARSE_TYPES
]

[18:38:43] WARNING: not removing hydrogen atom without neighbors
[18:38:43] WARNING: not removing hydrogen atom without neighbors
[18:39:42] WARNING: not removing hydrogen atom without neighbors
[18:39:42] WARNING: not removing hydrogen atom without neighbors
[18:40:42] WARNING: not removing hydrogen atom without neighbors
[18:40:42] WARNING: not removing hydrogen atom without neighbors
[18:41:42] WARNING: not removing hydrogen atom without neighbors
[18:41:42] WARNING: not removing hydrogen atom without neighbors
[18:42:42] WARNING: not removing hydrogen atom without neighbors
[18:42:42] WARNING: not removing hydrogen atom without neighbors


KeyboardInterrupt: 

In [ ]:
from rdkit.Chem.rdMolDescriptors import GetMACCSKeysFingerprint

MACCSKeys_rdkit_times = [
    get_times_rdkit(X, GetMACCSKeysFingerprint, sparse=sparse)
    for sparse in SPARSE_TYPES
]

In [ ]:
for sparse, i in enumerate(SPARSE_TYPES):
    plot_results(
        MACCSKeys_emf_times[i],
        MACCSKeys_rdkit_times[i],
        "MACCKeys fingerprint",
        count=None,
        sparse=sparse,
        save=False,
    )

## ERG Fingerprint

In [35]:
ERG_emf_times = [
    [
        get_times_emf(X, ERGFingerprint, n_jobs=n_cores, sparse=sparse)
        for n_cores in N_CORES
    ]
    for sparse in SPARSE_TYPES
]

KeyboardInterrupt: 

In [ ]:
from rdkit.Chem.rdReducedGraphs import GetErGFingerprint

ERG_rdkit_times = [
    get_times_rdkit(X, GetErGFingerprint, sparse=sparse)
    for sparse in SPARSE_TYPES
]

In [ ]:
for sparse, i in enumerate(SPARSE_TYPES):
    plot_results(
        ERG_emf_times[i],
        ERG_rdkit_times[i],
        "ERG fingerprint",
        count=None,
        sparse=sparse,
        save=False,
    )